# Asterisk

We propose Asterisk, a framework to generate high-quality training datasets at scale. Instead of relying on the end users to write user-defined heuristics, the proposed approach exploits a small set of labeled data and automatically produces a set of heuristics to assign initial labels. In this phase, the system applies an iterative process of creating, testing, and ranking heuristics in each, and every, iteration to only accommodate high-quality heuristics. Then, Asterisk examines the disagreements between these heuristics to model their accuracies. In order to enhance the quality of the generated labels, the framework improves the accuracies of the heuristics by applying a novel data-driven AL process. During the process, the system examines the generated weak labels along with the modeled accuracies of the heuristics to help the learner decide on the points for which the user should provide true labels. The process aims at enhancing the accuracy, and the coverage of the training data while engaging the user in the loop to execute the enhancement process. Therefore, by incorporating the underlying data representation, the user is only queried about the points that are expected to enhance the overall labeling quality. Then, the true labels provided by the users are used to refine the initial labels generated by the heuristics. 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import warnings
warnings.filterwarnings("ignore")


try:
    import heuristics_generator
except ImportError:
    print(ImportError)

Reading and preparing the data from csv file

In [ ]:
from heuristics_generator.hg_utils import *
DU= read_data("rain_prediction.csv", 'RainTomorrow')

In [ ]:
columns_list_full=['index', 'Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall',
       'Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am',
       'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainToday', 'RISK_MM', 'prediction']
df_features = DU
df_features = df_features.assign(Features = df_features.Date.astype(str)+
                             '@'+df_features.Location.astype(str)+
                             '@'+df_features.MinTemp.astype(str)+
                             '@'+df_features.MaxTemp.astype(str)+
                             '@'+df_features.Rainfall.astype(str)+
                             '@'+df_features.Evaporation.astype(str)+
                             '@'+df_features.Sunshine.astype(str)+
                             '@'+df_features.WindGustDir.astype(str)+
                             '@'+df_features.WindGustSpeed.astype(str)+
                             '@'+df_features.WindDir9am.astype(str)+
                             '@'+df_features.WindDir3pm.astype(str)+
                             '@'+df_features.WindSpeed9am.astype(str)+
                             '@'+df_features.WindSpeed3pm.astype(str)+
                             '@'+df_features.Humidity9am.astype(str)+
                             '@'+df_features.Humidity3pm.astype(str)+
                             '@'+df_features.Pressure9am.astype(str)+
                             '@'+df_features.Pressure3pm.astype(str)+
                             '@'+df_features.Cloud9am.astype(str)+
                             '@'+df_features.Cloud3pm.astype(str)+
                             '@'+df_features.Temp9am.astype(str)+
                             '@'+df_features.Temp3pm.astype(str)+
                             '@'+df_features.RainToday.astype(str)+
                             '@'+df_features.RISK_MM.astype(str))

df_ground = DU['prediction']
df_features = df_features.drop(columns_list_full, axis = 1)

In [ ]:
from heuristics_generator.hg_utils import *
doc_creation(df_features)

In [ ]:
global cand_extractor
T1, cand_extractor = cand_creation(df_features)
from snorkel import SnorkelSession
session = SnorkelSession()
rows = session.query(T1).count()

In [ ]:
train_sents, dev_sents, test_sents, train_ground, val_ground, test_ground = splitting_sets_with_labels(session, cand_extractor, T1, df_ground)

## Heuristics Generator
The system starts with the heuristics generator component which takes the labeled set DL and the unlabeled set DU as inputs and outputs a set of heuristics H of size K denoted as (h1, h2, …hk) and a vector of initial probabilistic labels for the points in DU.
The process starts with defining the input (features) for the potential models. Then, the process continues with creating the models (heuristics) and evaluating their performance and coverage. Finally, the process ranks the heuristics generated by each, and every, iteration to decide upon which heuristic to add to the set H.

In [ ]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""
columns_list= ['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall',
       'Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am',
       'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainToday', 'RISK_MM']

df_gold_label= prepare_gold_labels(session, T1, DU, columns_list)

When evaluating the performance of the heuristics produced during each iteration, the component also considers the overall coverage of the heuristics when applied to DU. The component aims to widen the range of the data points that receive labels from H in DU. In other words, the goal of the component is to output a set of heuristics that are individually accurate while achieving high labeling coverage when combined. Therefore, to estimate the performance of the heuristics. The performance metrics are computed by applying each heuristic to DL.

In [ ]:
from heuristics_generator.loader import DataLoader
dl = DataLoader()
train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, \
train_ground, val_ground, test_ground = dl.load_data_tabular(train_sents, test_sents, dev_sents, train_ground, test_ground, val_ground)

In [ ]:
from heuristics_generator.verifier import Verifier
from heuristics_generator.heuristic_generator import HeuristicGenerator
from heuristics_generator.synthesizer import Synthesizer

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")


hg = HeuristicGenerator(train_primitive_matrix, val_primitive_matrix, val_ground, train_ground, b=0.5)
hg.run_synthesizer(max_cardinality=1, idx=None, keep=6, model='lr')

syn = Synthesizer(val_primitive_matrix, val_ground, b=0.5)
heuristics, feature_inputs = syn.generate_heuristics('nn', 1)


In [ ]:
verifier = Verifier(hg.L_train, hg.L_val, val_ground, has_snorkel=False)
verifier.train_gen_model()
verifier.assign_marginals()

In [ ]:
feedback_idx = verifier.find_vague_points(gamma=0.1,b=0.5)

validation_accuracy = []
training_accuracy = []
validation_coverage = []
training_coverage = []
training_marginals = []
idx = None

hg = HeuristicGenerator(train_primitive_matrix, val_primitive_matrix, val_ground, train_ground, b=0.5)
for i in range(3,26):
    if i == 3:
        hg.run_synthesizer(max_cardinality=2, idx=idx, keep=3, model='dt')
    else:
        hg.run_synthesizer(max_cardinality=2, idx=idx, keep=1, model='dt')
    hg.run_verifier()
   
    va,ta, vc, tc = hg.evaluate()
    validation_accuracy.append(va)
    training_accuracy.append(ta)
    training_marginals.append(hg.vf.train_marginals)
    validation_coverage.append(vc)
    training_coverage.append(tc)
        
    hg.find_feedback()
    idx = hg.feedback_idx
    
    if idx == []:
        break

In [ ]:
training_marginals = training_marginals
L_train = hg.L_train

from scipy import sparse
L_train = L_train.astype(int)
L_train= sparse.csr_matrix(L_train) 

Finally, to combine the output of the heuristic and generate an initial vector of probabilistic labels, we employ a generative model to learn the accuracies of the heuristics in H and produce a single probabilistic label for each data point in the unlabeled dataset. 

In [ ]:
global gen_model 
gen_model, gen_train_marginals = Fitting_Gen_Model(L_train)

In [ ]:
predictions = gen_model.predictions(L_train, batch_size=None)
gen_model.learned_lf_stats()

In [ ]:
from heuristics_generator.hg_utils import *
abstain = 0
agreements_list=[]
for rows in L_train:
    if len(rows.data) == 0:
        abstain=abstain+1      
    agreements_list.append(abs(sum(rows.data)))

## Data-driven Active Learner
The component tries to integrate the user in the loop at this point by employing active learning. However, our problem settings do not impose traditional active learning scenarios where we usually have a small set of labeled points and a larger set of unlabeled data. Instead, we deal with a set of probabilistic labels that are classified based on the confidence of the generative model. Therefore, we adopt meta-active learning in this component and apply a data-driven approach to learn the query strategy. The approach formulates the process of designing the query strategy as a regression problem. We train a regression model to predict the reduction of the generalization error associated with adding a labeled point {xi, yi} to the training data of a classifier. Our main hypothesis is that this regressor can serve as the query strategy in our problem settings to outperform the baseline strategies since it is customized to the underlying distribution and considers the output of the generative model.

In [ ]:
index_list = []
agreements_list=[]
unlabeled_list=[]
rows = session.query(T1).filter(T1.split==0)
for r in rows: 
    index_list.append(r.Features.sentence.document_id-1),
for rows in L_train:
    if len(rows.data) == 0:
        unlabeled_list.append(True)
    else:
        unlabeled_list.append(False)
    agreements_list.append(abs(sum(rows.data)))
    
df_with_additional_info = pd.DataFrame(index_list)
df_with_additional_info.columns=['index']
df_with_additional_info['disagreement_factor'] = agreements_list
df_with_additional_info['unlabeled_flag'] = unlabeled_list

In [ ]:
df_with_additional_info=df_with_additional_info.rename(columns={'index': 'index'})
original_data = DU
original_data = DU.rename(columns={'Index': 'index'})
df_with_additional_info = df_with_additional_info.merge(original_data, on=['index'], how='left', indicator= True)
cond1 = df_with_additional_info['unlabeled_flag'] == True
cond2 = df_with_additional_info['disagreement_factor'] < 3
df_active_learning= df_with_additional_info[cond1 | cond2]
print("Data with additional info: disagreements and abstain")
print(df_with_additional_info.shape)
print("Data for Active learning: Data with additional info after applying conditions")
print(df_active_learning.shape)
df_active_learning = df_active_learning.drop(['unlabeled_flag', 'disagreement_factor', '_merge'], axis=1)

In [ ]:
budget_variable=0.7
labeling_budget = int(budget_variable*len(df_with_additional_info.index))
if(labeling_budget >= df_active_learning.shape[0]):
    labeling_budget= int(1*len(df_active_learning.index))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from data_learner.active_learner import *
from data_learner.models import DDL_Dataset 
from data_learner.lal_model import LALmodel

fn = 'LAL-iterativetree-simulatedunbalanced-big.npz'
parameters = {'est': 1000, 'depth': 40, 'feat': 6 }
filename = '../data_learner/datasets/'+fn
regression_data = np.load(filename)
regression_features = regression_data['arr_0']
regression_labels = regression_data['arr_1']
lalModel = RandomForestRegressor(n_estimators = parameters['est'], max_depth = parameters['depth'], 
                                 max_features=parameters['feat'], oob_score=True, n_jobs=8)
lalModel.fit(regression_features, np.ravel(regression_labels))    

In [ ]:
from data_learner.dl_utils import *
indices, labels= run_dll(lalModel, labeling_budget,df_active_learning);

In [ ]:
indices

## Probabilistic Labels Generator
The final component of Asterisk is the label generator, which aims at learning the accuracies of the generated heuristics using the refined heuristics matrix Hupdated, and then combines all the output of these heuristics to produce a single probabilistic label for each point in DU. This process is accomplished by learning the structure of a generative model Gen which utilizes the refined matrix to model a process of labeling the training set.

In [ ]:
AL_Results=pd.DataFrame(columns=['index','AL_Label'])
AL_Results['index']=alLALiterative.indices.astype('int64')
AL_Results['AL_Label']=labels
AL_Results = AL_Results.sort_values(by =['index'])

In [ ]:
AL_Results.loc[AL_Results['AL_Label']==0, 'AL_Label']=-1
data_with_AL_Results = df_with_additional_info.merge(AL_Results, on=['index'], how='left')
true_label = data_with_AL_Results['AL_Label']
true_label = true_label.fillna(0)
for i in range(len(true_label)):
    if true_label[i] !=0:
        L_train[i,:]=true_label[i]
gen_model, AL_train_marginals = Fitting_Gen_Model(L_train)
predictions = gen_model.predictions(reef_L_train, batch_size=None)


In [ ]:
import matplotlib.pyplot as plt
plt.hist(predictions, bins=20)
plt.show()